# Algoritmo Evolutivo para la optimización de los parámetros en un problema de Aprendizaje Computacional

Problema de los dígitos

In [ ]:
import numpy as np
from sklearn.datasets import load_digits

X,Y = load_digits(n_class=10, return_X_y=True)
print(X.shape)
print(len(Y))

In [ ]:
import matplotlib.pyplot as plt

def print_image( sample,y ):
    img = np.reshape(sample,(8,8))
    print('Digit',y)
    plt.imshow(img, cmap = 'Greys')
    plt.show()

index = 198
print_image(X[index,:],Y[index])
print(np.unique(Y,return_counts=True))

## Clasificación

In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, Ytrain, Ytest = train_test_split( X, Y, test_size=0.30, random_state=0)
print('Train:',Xtrain.shape,len(Ytrain))
print('Test:',Xtest.shape,len(Ytest))

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

model = KNeighborsClassifier(  )
model.fit(Xtrain,Ytrain)
Yc = model.predict(Xtest)
print(accuracy_score(Ytest,Yc))


# Solución con un algoritmo evolutivo

Configuración

In [ ]:
import sklearn
models = {
    'sklearn.tree.DecisionTreeClassifier': {
        'integer_parameters':{
            'max_depth': [5, 100]
        },
        'continuous_parameters':{ },
        'discrete_parameters':{
            'criterion': ["gini", "entropy"]
        }        
    },
    'sklearn.neighbors.KNeighborsClassifier': {
        'integer_parameters':{
            'n_neighbors': [1, 20]
        },
        'continuous_parameters':{  },
        'discrete_parameters':{
            'weights': ["uniform", "distance"],
            'p': [1, 2]
        }
        
    },
    'sklearn.svm.SVC': {
        'integer_parameters':{  },
        'continuous_parameters':{
            'C': [1e-4, 25.]
        },
        'discrete_parameters':{
            'kernel':['linear','poly','rbf','sigmoid']
        }
    },
    'sklearn.neural_network.MLPClassifier':{
        'integer_parameters':{
            'hidden_layer_sizes': [1,200] #(100,)
        },
        'continuous_parameters':{ },
        'discrete_parameters':{
            'activation':['relu','identity','logistic','tanh']
        }
    }

}

## Representación de los individuos

In [ ]:
individual = ['sklearn.svm.SVC',['C','kernel'],[1.0,'linear'],['continuous','discrete']]

smodel = individual[0]+'('
for i in range(len(individual[1])):
    if isinstance(individual[2][i],str):
        smodel+=individual[1][i]+'='+"'"+individual[2][i]+"'"
    else:
        smodel+=individual[1][i]+'='+str(individual[2][i])
    smodel+=','
smodel=smodel[:-1]+')'
print(smodel)

In [ ]:
program = "import sklearn \n"
program+= "model = "+smodel +"\n"
print(program)

d = {}
exec(program,d)
model = d['model']

print(model)
model.fit(Xtrain,Ytrain)
Yc = model.predict(Xtest)
acc = accuracy_score(Ytest,Yc)
print(acc)

## Cálculo de la aptitud

In [ ]:
def fitness_individual(individual):
    smodel = individual[0]+'('
    for i in range(len(individual[1])):
        if isinstance(individual[2][i],str):
            smodel+=individual[1][i]+'='+"'"+individual[2][i]+"'"
        else:
            smodel+=individual[1][i]+'='+str(individual[2][i])
        smodel+=','
    smodel=smodel[:-1]+')'

    program = "import sklearn \n"
    program+= "model = "+smodel +"\n"
    d = {}
    exec(program,d)
    model = d['model']
    model.fit(Xtrain,Ytrain)
    Yc = model.predict(Xtest)
    acc = accuracy_score(Ytest,Yc)
    return acc

#individual = ['sklearn.neighbors.KNeighborsClassifier',['n_neighbors','weights','p'],[20,'uniform',2]]
individual = ['sklearn.svm.SVC',['C','kernel'],[1.0,'linear']]
fit = fitness_individual(individual)
print('fitness:',fit)


## Población inicial

Obtener los valores de la configuración de los modelos

In [ ]:
nmodels = len(models)
print('keys:',models.keys() )
print('keys list:',list( models.keys() ) )
print()

classifier = list( models.keys())[ np.random.randint(nmodels)  ] 
print('Random element:',classifier)
print()

print('  Integer parameters:', list(models[classifier]['integer_parameters'].keys()) )
print('  Continuous parameters:', list(models[classifier]['continuous_parameters'].keys()) )
print('  Discrete parameters:', list(models[classifier]['discrete_parameters'].keys()) )           

In [ ]:
for parameter in list(models[classifier]['integer_parameters'].keys()):
    values = models[classifier]['integer_parameters'][parameter]
    print(parameter,'-', values)
    
for parameter in list(models[classifier]['continuous_parameters'].keys()):
    values = models[classifier]['continuous_parameters'][parameter]
    print(parameter,'-', values)
    
for parameter in list(models[classifier]['discrete_parameters'].keys()):
    values = models[classifier]['discrete_parameters'][parameter]
    print(parameter,'-', values)
    

Crear un individuo aleatorio

In [ ]:
def random_individual(models):
    classifier = list( models.keys())[ np.random.randint(len(models))  ]
    individual = [ classifier ] 
    parameters_names = []
    parameters_values = []
    parameters_types = []
    
    for parameter in list(models[classifier]['integer_parameters'].keys()):
        values = models[classifier]['integer_parameters'][parameter]
        parameters_names.append(parameter)
        parameters_values.append( np.random.randint(values[0],values[1]+1))
        parameters_types.append( 'integer' )
        
    for parameter in list(models[classifier]['continuous_parameters'].keys()):
        values = models[classifier]['continuous_parameters'][parameter]
        parameters_names.append(parameter)
        parameters_values.append( np.random.uniform(values[0],values[1]))
        parameters_types.append( 'continuous' )
    
    for parameter in list(models[classifier]['discrete_parameters'].keys()):
        values = models[classifier]['discrete_parameters'][parameter]
        parameters_names.append(parameter)
        parameters_values.append( values[np.random.randint(len(values))])
        parameters_types.append( 'discrete' )
    
    individual.append(parameters_names)
    individual.append(parameters_values)
    individual.append(parameters_types)
    return individual

individual = random_individual(models)
print(individual)


In [ ]:
def initial_population(N,models):
    population = []
    for i in range(N):
        population.append(random_individual(models))
    return population

N = 5
population = initial_population(N,models)
print(population)
fitness = [ fitness_individual(ind) for ind in population ]
print(fitness)

## Mutation

In [ ]:
def mutation(ind):
    pass  
    return nind    

## Selection

In [ ]:
def tournament(population,fitness,tournament_size = 2,negative=False):
    idx = []
    fit = []
    for t in range(tournament_size):
        i = np.random.randint( len(population) )
        idx.append(i)
        fit.append(fitness[i])
    if negative:
        m = np.argmin(fit)
        return idx[m]
    else:
        m = np.argmax( fit )
        m = idx[m]
        return population[m],fitness[m]

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
N = 10
K = 50
population = initial_population(N,models)
fitness = [ fitness_individual(ind) for ind in population ]
elite = np.argmax(fitness)
print(0, fitness[elite],population[elite])

for k in range(1,K+1):
    ind,fit = tournament(population,fitness)
    indm = mutation(ind)
    fitm = fitness_individual(indm)
    
    i = tournament(population,fitness,negative=True)
    population[i] = indm
    fitness[i] = fitm
    
    # Print
    print('\n')
    elite = np.argmax(fitness)
    print(k, fitness[elite],population[elite])
    for i in range(N):
        print('  ',i,fitness[i],population[i][0])
print('\n')
print('Best program')
print(fitness[elite],population[elite])
